In [1]:
import numpy as np 
import pandas as pd
import statsmodels.api as smf # python linear regression package! 
import matplotlib.pyplot as plt

### Calculating the OLS Estimator

In this notebook, we will calculate the **ordinary least squares (OLS) estimator**, along with other important statistical measures using our very own model built from scratch in Python! 

Ordinary least squares is a classic and often-used linear least-squares method for estimating parameters in a linear regression model. For example, let's suppose there are $n$ observations, $(x_{i}, y_{i})$, where $x_{i}$ is a vector of $r$ regressors such that $x_{i} = [x_{i1}, x_{i2}, ..., x_{ir}]$. 

Then, we can write $y_{i}$ as a function of the regressors in order to obtain a linear model: 

$$ y = x^{T}\beta + \varepsilon$$ 

where $\beta$ is a $r\times1$ vector of parameters that we estimate via the ordinary least squares, $y$ and $\varepsilon$ are $n\times1$ vectors of the outcome variables (predicted) and the errors. $x$ is an $n\times p$ matrix of the regressors. The estimate of the regression parameters, which is denoted as $\hat{\beta}$, is called the ordinary least squares estimator, and is given by the following equation:  

$$\hat{\beta} = (x^{T}x)^{-1}x^{T}y$$

Now, let's put this concept into practice by finding the OLS estimate for a sample regression! We will be using the same dataset as in the Ommitted Variable Bias notebook, with logwage as our outcome variable of interest, and state, age, education, and gender as regressors:

Thus, our regression model is: 

$$ log(wage_{i}) = \beta_{1}age_{i} + \beta_{2}educ_{i} + \beta_{3}female_{i} + \varepsilon_{i}$$

We can estimate the parameters $\beta$ with the statsmodels API as below: 

In [19]:
# read in data
ovb_data = pd.read_csv('ovb.csv')
ovb_data

,state,age,wagesal,imm,hispanic,black,asian,educ,wage,logwage,female,fedwkr,statewkr,localwkr
0,11,44,18000,0,0,0,0,14,9.109312,2.209297,1,1,0,0
1,11,39,18000,0,0,0,0,14,18.000000,2.890372,0,0,0,0
2,11,39,35600,0,0,0,0,12,17.115385,2.839978,0,0,0,1
3,11,39,8000,0,0,0,0,14,5.128205,1.634756,1,0,0,0
4,11,39,100000,0,0,0,0,16,38.461538,3.649659,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21902,95,36,125000,0,0,0,0,18,60.096154,4.095946,0,0,1,0
21903,95,38,70000,0,0,0,1,18,26.923077,3.292984,1,0,0,0
21904,95,43,48208,0,0,0,0,14,20.601709,3.025374,1,0,0,0
21905,95,43,75000,0,0,0,0,18,36.057692,3.585120,0,0,0,0


In [39]:
# define y and x sets
y = ovb_data['logwage'] 

# female is already a dummy variable that we can use. If there was also a male = 1 - female dummy variable, 
# then we would have to drop it from the regression due to perfect multicollinearity which would make the 
# inverse of the regressor matrix (design matrix) impossible to find. 
X = ovb_data[['age', 'educ', 'female']] 

In [48]:
# calculating the estimated coefficients, using the OLS estimator equation above
# NOTE: must convert Pandas Series to numpy arrays first

def ols_estimator(X, y):
    return np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)

beta_hats = ols_estimator(np.array(X), np.array(y))

regression_results = pd.DataFrame(index = ['age', 'educ', 'female'], 
                                      columns = ['Beta estimates'], 
                                      data=beta_hats)
regression_results

,Beta estimates
age,0.036325
educ,0.119588
female,-0.281464


### Calculating Variance and Standard Errors of Each Coefficient

Suppose we are interested in studying not only what the estimated coefficients on our regressors are, but also how "good" of a fit these estimators provide. One way that we can do this is my looking at the variance of our OLS estimator and the standard errors.

Assuming homoscedasticity, the variance of the OLS estimator is given by: 

$$Var(\hat{\beta}) = \hat{\sigma}^{2}_{\varepsilon}(x^{T}x)^{-1}$$

where $\hat{\sigma}^{2}_{\varepsilon}$ is the variance of the error term. How do we find the variance of this error term? Well, in linear regression, the unbiased estimator of the error term is given by

$$ \hat{\sigma}^{2}_{\varepsilon} = \frac{1}{n-p-1}\sum_{i=1}^{n}{(y_{i} - \hat{y}_{i})^{2}} = \frac{1}{n-p-1}\sum_{i=1}^{n}{\hat{\varepsilon}_{i}} $$

where $n$ is the number of observations in the sample and $p$ is the number of regressors. It should also be noted that the summation term in the equation above is known as the **sum of squared residuals**.

Thus, $Var(\hat{\beta})$ then becomes a square matrix of shape $r \times r$, also known as the variance-covariance matrix. One property of this matrix is that it provides the covariance between pairs of elements in a random vector. Therefore, the matrix provided by $Var(\hat{\beta})$ contains the covariance between elements of $\hat{\beta}$. Furthermore, in order to find the **variance** of each individual coefficient, we must take each values on the diagonal of this matrix, since $Cov(\hat{\beta}_{i}, \hat{\beta}_{i}) = Var(\hat{\beta}_{i})$. To find the **standard error** of these values, we simply take the square root of the variance.

Let's try to calculate these values below using our data and OLS coefficients.

In [44]:
def error_term_variance(n, p, X, beta_hats, y): 
    # calculate the predicted value of y
    pred_y = np.dot(X, beta_hats) 
    # calculate the sum of squared residuals
    SSR = np.sum((y - pred_y)**2)
    # calculate the variance of the error term 
    return SSR/(n-p-1)

def variance_covariance_matrix(sigma_hat, X): 
    return sigma_hat * np.linalg.inv(np.dot(X.T, X))

n = X.shape[0]
p = X.shape[1]

sigma_hat = error_term_variance(n, p, np.array(X), beta_hats, np.array(y))
var_beta_hat = variance_covariance_matrix(sigma_hat, np.array(X))

print(var_beta_hat)

[[ 2.34945628e-07 -6.19448724e-07 -5.29988351e-07]
 [-6.19448724e-07  1.78907402e-06 -8.11555059e-07]
 [-5.29988351e-07 -8.11555059e-07  6.70192918e-05]]


Above is the displayed variance-covariance matrix -- taking the square root of values along the diagonal will provide us with the standard error for each term: 

In [55]:
def coeff_var_from_var_matrix(matrix): 
    return [matrix[i][i] for i in range(len(matrix))]

def coeff_se_from_var_matrix(matrix): 
    return [np.sqrt(matrix[i][i]) for i in range(len(matrix))]

variances = coeff_var_from_var_matrix(var_beta_hat)
standard_errors = coeff_se_from_var_matrix(var_beta_hat)

# add to the regression results!
regression_results['Variances'] = variances
regression_results['Standard Errors'] = standard_errors
regression_results

,Beta estimates,Standard Errors,Variances
age,0.036325,0.000485,2.349456e-07
educ,0.119588,0.001338,1.789074e-06
female,-0.281464,0.008187,6.701929e-05


### (In Progress) Finding the t-statistic and confidence interval

### (In Progress) Calculating R-squared and Adjusted R-squared

### The Easier Way: Statsmodels API

Great! now that we have found the estimates for each coefficient, we can confirm the accuracy of our calculations using the statsmodels API. 

Using the statsmodels API, we can easily find the OLS regression estimates with a couple lines of code -- along with some extra statistics that are useful when conducting a regression analysis. 

In [17]:
model1 = smf.OLS(y, X) 
res1 = model1.fit()
print(res1.summary())

                                 OLS Regression Results                                
Dep. Variable:                logwage   R-squared (uncentered):                   0.961
Model:                            OLS   Adj. R-squared (uncentered):              0.961
Method:                 Least Squares   F-statistic:                          1.797e+05
Date:                Thu, 04 Nov 2021   Prob (F-statistic):                        0.00
Time:                        17:09:29   Log-Likelihood:                         -20071.
No. Observations:               21907   AIC:                                  4.015e+04
Df Residuals:                   21904   BIC:                                  4.017e+04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Nice! We can see that the estimated regression coefficients are shown in the table above, along with some relevant statistics such as $R^{2}$, standard error, the t-statistic, and the confidence interval for each coefficient. 

### (In Progress) Interpreting the coefficients

### Extra 1: A note on adding dummy variables

### Extra 2: Regressions with just a constant